# Transfer Learning

In [1]:
# Algo qui classe hauteur, longueur, profondeur
# print(model.predict) TEST 
# print(labels)
# principe précision rappel -> courbe PR

In [2]:
## Fonction alerte vocale à la fin d'un programme
def Bipper():
    duration = 0.3  # second
    freq = 300  # Hz
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))
    os.system('say "program finished"')

## PARTIE 1 : Création base de données (pandas et array)

In [8]:
## Créer la base de données database et database_array, Obtenir les données texte (Excel)

# définir les chemins utiles
#path_data = "/Users/hanae/Documents/2_Formation/CentraleSupélec/3A/3_Projet/Data"
path_data="/Users/Claire/Documents/1SUPELEC/3A/Projet"

### A ADAPTER, chemin aux données images
path_excel = os.path.join(path_data,"STIC_IGR_infos_patients_v_simple_ID.xlsx")
### A ADAPTER, chemin à l'excel
path_images_main = os.path.join(path_data,"images_anonymises_projet_3A")
### A ADAPTER, chemin au dossier principal images
folders = [f for f in os.listdir(path_images_main) if not f.startswith('.')] # lister les sous-dossiers
# de la base de données "STIC_XXX" sans prendre en compte les fichiers cachés commençants par '.'

# créer la base de données pandas et array
Mon_Excel = open(path_excel,'r') # ouvrir l'excel en mode "read"
database = pd.read_excel(path_excel) # lire et former une base de données Pandas à partir de l'excel

# rajouter la variable correspondant au volume
Local_index = database.index
Local_columns = database.columns
Local_columns = ['Volume'] + pd.Index.tolist(Local_columns)
Volume = database['Length'].values*database['Thickness'].values*database['Width'].values
database = database.values
database = np.concatenate((database,Volume.reshape(-1,1)), axis=1)
database = pd.DataFrame(database, index=Local_index, columns=Local_columns)
database_array = database.values # créer une variable avec la base de données en numpy array pour travailler
Mon_Excel.close() # fermer l'excel

# paramètres
N = len(database_array) # N = nombre de patients (179)
LINES_index = [database_array[i][0] for i in range(N)] # liste des ID patients : "Patient 0"
COLUMNS_index = ["ID","Pathology","Localisation","Length","Width","Thickness","Sex","Progression","Volume"]
# paramètres initiaux
nb_parameters = len(COLUMNS_index)

In [9]:
## Extraire les features pour chaque image

# Télécharger le modèle ResNet50 de Keras qui servira à extraire les features des images
model = ResNet50(weights='imagenet', include_top=False)

# Initialiser les matrices de features de la 1ère image et de la 2ème si elle existe pour les N patients
nb_features = 2048 # 2048 features par image
Features_img_1 = np.ndarray((N,nb_features))
Features_img_2 = np.ndarray((N,nb_features))

for index in range(N): # pour chaque patient
    print('{}/{}'.format(index+1, N)) # afficher sur quel patient on travaille
    patient_id = LINES_index[index] # récupérer l'ID du patient, chaîne de caractères "Patient 0"

    for folder in folders: # on parcourt chaque sous-dossier images
        images = []
        patient_nb = str.split(patient_id, " ") # on sépare les 2 mots de l'ID patient : "Patient" et "0"
        images = [file for file in os.listdir(os.path.join(path_images_main, folder))
                 if file.endswith('dcm') and file.startswith(patient_nb[0] + '_' + patient_nb[1] + '.')]
        # on récupère les fichiers images en .dcm qui commencent par "Patient_0."
        images_1 = []
        images_2 = []
        
        # Traitement de la 1ere image
        if images != []: # si l'image existe
            print(images) # on affiche les fichiers images trouvés pour être sure que tout va bien
            images_1 = [images[0]] # on récupère la 1ère image
            for image in tqdm(images_1) : # tqdm permet d'afficher la barre de progression
                img = pydicom.dcmread(os.path.join(path_images_main, folder, image)) # lire l'image
                print(img.pixel_array.shape) # on affiche la taille de l'image pour vérifier

                # Récupération des features à l'aide du ResNet50
                img = img.pixel_array # on lui donne ce qu'on veut tant que c'est carré
                # dim de l'image (x,y,channels (=3))
                size_x = len(img)
                size_y = len(img[0])
                listImages = np.zeros((1, size_x, size_y, 3))
                listImages[0,:,:,:] = img # on reformate l'image en rajoutant une dimension au début XXX
                y = model.predict(listImages) # extraction des features
                y = y.mean(1).mean(1) # XXX
                y = y.flatten() # on reformate les features pour avoir une liste
                Features_img_1[index] = y

            # Traitement de la 2nde image
            if len(images) > 1:
                images_2 = [images[1]]
                for image in tqdm(images_2) :
                    img = pydicom.dcmread(os.path.join(path_images_main, folder, image))
                    print(img.pixel_array.shape)

                    # Récupération des features à l'aide du ResNet50
                    img = img.pixel_array # on lui donne ce qu'on veut tant que c'est carré
                    size_x = len(img)
                    size_y = len(img[0])
                    listImages = np.zeros((1, size_x, size_y, 3))
                    listImages[0,:,:,:] = img
                    y = model.predict(listImages)
                    y = y.mean(1).mean(1)
                    y = y.flatten()
                    Features_img_2[index] = y

Bipper() # alerte vocale

C:\Users\Claire\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


1/179
['Patient_0.Seq5.Ser1.Img17.dcm', 'Patient_0.Seq5.Ser1.Img18.dcm']


  0%|          | 0/1 [00:00<?, ?it/s]

(376, 592, 3)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

(378, 563, 3)


  0%|          | 0/1 [00:00<?, ?it/s]

(378, 563, 3)


100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


In [6]:
## rajouter les 2048 features de chacune des 2 images à la base de données initiale  

# rajouter des colonnes
for i in range(nb_features):
    COLUMNS_index += ['Feature Img1 %s' % (i+1)] # RQ : i+1 pour commencer à 1
for i in range(nb_features):
    COLUMNS_index += ['Feature Img2 %s' % (i+1)]

database_array = np.concatenate((database_array, Features_img_1, Features_img_2), axis=1)
# on ajoute les features des images à la base de données
database_transfer = pd.DataFrame(database_array, columns=COLUMNS_index, index=LINES_index)
# on crée une nouvelle base de données Pandas
database_transfer_copy = database_transfer # on crée une copie par sécurité

# visualiser la nouvelle base de données (vérifier que les features sont non nulles sauf pour quelques patients)
database_transfer

,ID,Pathology,Localisation,Length,Width,Thickness,Sex,Progression,Volume,Feature Img1 1,...,Feature Img2 2039,Feature Img2 2040,Feature Img2 2041,Feature Img2 2042,Feature Img2 2043,Feature Img2 2044,Feature Img2 2045,Feature Img2 2046,Feature Img2 2047,Feature Img2 2048
Patient 0,Patient 0,Other,Péritonéale 1,55,74,24,Male,1,97680,0.955071,...,0.0299494,0.198278,0.258754,0.019006,0.241954,0,0,1.10882,0,0.321722
Patient 1,Patient 1,Other,Hépatique segment 5,29,33,23,Male,0,22011,0.845327,...,0.0347256,0.0355004,0.508187,0.0314798,0.1434,0.0223533,0.0517525,1.03143,0.00239146,1.08519
Patient 2,Patient 2,Melanoma,Ganglionnaire Inguinale,37,40,25,Male,1,37000,0,...,0,0,0,0,0,0,0,0,0,0
Patient 3,Patient 3,Colorectal Cancer,Hépatique segment 7,39,50,33,Male,1,64350,0.45934,...,0.0203768,0.12252,0.395684,0.108101,0.273029,0.00751764,0.0648635,1.43971,0.0975807,1.24903
Patient 4,Patient 4,Other,Hepatique segment 4,80,63,60,Female,1,302400,0.551654,...,0.0534588,0.177757,0.210149,0,0.294201,0.0160407,0.00141275,0.837975,0.0243745,2.08142
Patient 5,Patient 5,Other,Péritonéale 7,30,36,12,Male,1,12960,0.382868,...,0.0892332,0.00583194,0.183723,0.0325058,0.140609,0.0135138,0.0016169,2.09385,0.039209,0.0488435
Patient 6,Patient 6,Other,Péritonéale 1,22,22,18,Male,1,8712,0.699323,...,0.118895,0.0128383,0.146437,0.00390649,0.341562,0.0127877,0.00286831,2.96362,0.0595357,0.0777456
Patient 7,Patient 7,Other,Péritonéale 2,89,95,44,Male,1,372020,0.831501,...,0.00115698,0.512618,0.318428,0.034494,0.21479,0.0845777,0.0166887,0.653875,0.011894,2.03709
Patient 8,Patient 8,Other,Péritonéale 7,25,24,20,Female,1,12000,0.752031,...,0.216204,0.00935275,0.236889,0.00213906,0.298514,0.00552384,0.00588984,1.60742,0.0728894,0.20121
Patient 9,Patient 9,Other,Péritonéale 4,72,47,42,Male,1,142128,0.818561,...,0.0548724,0.166488,0.407924,0.0138262,0.418861,0.0168091,0.00376299,1.35728,0.017236,1.02391


## PARTIE 2 : Préparation et nettoyage des données

In [7]:
## Quels patients sont non conformes qui doivent être écartés ?

# QUESTION : il s'agit des ID patients non anonymisés :/
### VIRER A LA MAIN IMAGE SPECIFIQUE OU PATIENT EN ENTIER PLUS SIMPLE
to_delete = ['200909232HB','200315951GW','200705382HW','200708823MG','200804172EX','200805199SH','200805583NW',
            '200901284GS','200902380ES','7002641BA','200700186ET'] # images problème détection de contours 
print(len(to_delete))
print(to_delete)

# Problème : Le 200805989BD a les 2 sets d'image mais avec un nom ID_jour_mois_année[...].dcm.
# Idem pour 200503135XF mais le nom est le même, c'est juste que les img sont suffisemment différentes.
# Il conviendrait alors de refaire le travail depuis le début en faisant une sélection de l'image
# sur la date de base line (contenu dans le fichier dicom).
# -> dans un premier temps : juste supprimer ces patients.

11
['200909232HB', '200315951GW', '200705382HW', '200708823MG', '200804172EX', '200805199SH', '200805583NW', '200901284GS', '200902380ES', '7002641BA', '200700186ET']


In [8]:
##  Mais alors combien avons-nous de patients différents ? 

# QUESTION : pas le même nombre de patients (167 pour Emmanuel)
# FAIRE ATTENTION AUX 4 PATIENTS AVEC 2 traitements d'immuno à 6 mois d'écart

patients = [] # liste initialisée de patients uniques
for folder in folders: # pour chaque dossier
    for file in os.listdir(os.path.join(path_images_main, folder)): # on parcourt la liste des images
        patient_ID = str.split(file, ".") # séparer 'Patient_4.Seq1.Ser1.Img11.dcm' en
        # 'Patient_4', 'Seq1', 'Ser1', 'Img11', 'dcm'
        patient_ID = patient_ID[0] # on récupère 'Patient_4'
        patient_ID = str.split(patient_ID, "_") # on sépare 'Patient' et '4'
        patient_ID = patient_ID[1] # on ne garde que '4'
        if patient_ID not in patients:
            patients += [patient_ID] # on l'ajoute à la liste que s'il est unique
print(len(patients))

163


In [9]:
## Nettoyage : Retirer les patients sans images (appelés intrus) et les doublons

# QUESTION : pas le même nombre d'intrus (28 pour Emmanuel)

COLUMNS = database_transfer.columns
FEATURES_img_1_index = COLUMNS[nb_parameters:nb_parameters+nb_features] # colonnes features de l'image 1
FEATURES_img_2_index = COLUMNS[nb_parameters+nb_features:nb_parameters+2*nb_features] # colonnes features image 2
intrus = to_delete.copy() # posent problème détection de contours

for patient in LINES_index:
    booleen = True
    for feat in FEATURES_img_1_index : # Traitement des pbs de l'img 1
        if type(database_transfer.loc[patient][feat]) == pd.core.series.Series : # si type(feature) = liste pandas
            if database_transfer.loc[patient][feat].values[0] > 0.000001: # on teste si sa 1ère valeur
                booleen = False
            break # ainsi on vire tous les doublons
        else :
            if database_transfer.loc[patient][feat] > 0.000001 :
                booleen = False
        if booleen == False :
            break
    for feat in FEATURES_img_2_index : # Traitement des pbs de l'img 2
        if type(database_transfer.loc[patient][feat]) == pd.core.series.Series :
            if database_transfer.loc[patient][feat].values[0] > 0.000001:
                booleen = False
            break # ainsi on vire tous les doublons
        else :
            if database_transfer.loc[patient][feat] > 0.000001 :
                booleen = False
        if booleen == False :
            break
    if booleen == True and patient not in intrus :
        intrus += [patient]
print("nb_intrus :",len(intrus))
print("les intrus sont :",intrus[len(to_delete):])
for element in intrus:
    if element in database_transfer.index :
        database_transfer = database_transfer.drop(element,axis=0)

patients_concerned = database_transfer.index

Bipper() # alerte vocale

nb_intrus : 28
les intrus sont : ['Patient 2', 'Patient 20', 'Patient 33', 'Patient 36', 'Patient 37', 'Patient 40', 'Patient 43', 'Patient 56', 'Patient 60', 'Patient 68', 'Patient 84', 'Patient 88', 'Patient 95', 'Patient 132', 'Patient 159', 'Patient 171', 'Patient 176']


In [10]:
## Combien de colonnes sont textuelles et poseront problème ?
Column_txt = []
for elt in COLUMNS:
    if type(database_transfer_copy[elt][0]) == str:
        Column_txt += [elt]
Column_txt
Column_txt = []
for elt in COLUMNS:
    if type(database_transfer[elt][0]) == str:
        Column_txt += [elt]
print("Colonnes de textes : ", Column_txt)

Colonnes de textes :  ['ID', 'Pathology', 'Localisation', 'Sex']


In [11]:
## Recalibrage des variables
# les algos de SL de marchent pas très bien lorsque les variables ont des échelles trop différentes

standardiser = StandardScaler() # à voir, plusieurs fois dans le pipeline améliore les choses
first_encoder = LabelEncoder() # obligé de faire un premier encodage car OneHot marche aux nombres
# pour les valeurs textuelles, tranformer les mots en nombre
second_encoder = OneHotEncoder() # evite de biaiser l'algo

# pour plus tard il faut sauvegarder database_transfer["Pathology"]
# pour avoir un train et un test sets homogène
Label_shufflesplit = database_transfer["Pathology"].copy()

for elt in Column_txt:
    database_transfer_cat = database_transfer[elt]    
    database_transfer_cat_encoded = np.ndarray.tolist(first_encoder.fit_transform(database_transfer_cat))
    for i in range(len(database_transfer_cat_encoded)):
        database_transfer_cat_encoded[i] = [database_transfer_cat_encoded[i]]
    database_transfer_cat_encoded = np.array(database_transfer_cat_encoded)
    database_transfer_cat_encoded_final = database_transfer_cat_encoded
    database_transfer_cat_encoded_final = second_encoder.fit_transform(database_transfer_cat_encoded_final).toarray()

    for i in range(len(database_transfer[elt])):
        database_transfer[elt][database_transfer.index[i]] = standardiser.fit_transform(database_transfer_cat_encoded_final[i].reshape(-1,1))

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto

In [12]:
## Recalibrage des variables
# les algos de SL de marchent pas très bien lorsque les variables ont des échelles trop différentes

database_transfer_save = database_transfer

# Mieux de normaliser pour être moins sensible aux valeurs extrêmes que transfo min-max
standardiser = StandardScaler()

# Attention, toutes les variables ne sont pas à normaliser!
# Ne pas prendre en compte les variables textuelles et binaires (progression)
Pathology = database_transfer["Pathology"].values
database_transfer = database_transfer.drop("Pathology",axis=1)
Localisation = database_transfer["Localisation"].values
database_transfer = database_transfer.drop("Localisation",axis=1)
Sex = database_transfer["Sex"].values
database_transfer = database_transfer.drop("Sex",axis=1)
Progression = database_transfer["Progression"].values
database_transfer = database_transfer.drop("Progression",axis=1)
ID = database_transfer["ID"].values
database_transfer = database_transfer.drop("ID",axis=1)

Pathology = Pathology.reshape(-1,1)
Localisation = Localisation.reshape(-1,1)
Sex = Sex.reshape(-1,1)
Progression = Progression.reshape(-1,1)
ID = ID.reshape(-1,1)

In [13]:
database_transfer = standardiser.fit_transform(database_transfer.values)
database_transfer = np.concatenate((ID,Pathology,Localisation,Sex,Progression,database_transfer),axis=1)
COLUMNS_index = ['ID','Pathology','Localisation','Sex','Progression','Length','Width','Thickness','Volume']
for i in range(1,2049):
    COLUMNS_index += ['Feature Img1 %s' % (i)]
for i in range(1,2049):
    COLUMNS_index += ['Feature Img2 %s' % (i)]
    
database_transfer = pd.DataFrame(database_transfer,index=patients_concerned,columns=COLUMNS_index)

# Retour à la normale après la manipulation du fichier précédent
# Obligé de faire une copie sinon il refait le StandardScaler tout seul
# taille_Pathology=len(STIC_Transfer['Pathology']['200610546EP'])
# taille_Localisation=len(STIC_Transfer['Localisation']['200610546EP'])
# taille_Sex=len(STIC_Transfer['Sex']['200610546EP'])
taille_Pathology = len(database_transfer['Pathology']['Patient 0'])
taille_Localisation = len(database_transfer['Localisation']['Patient 0'])
taille_Sex = len(database_transfer['Sex']['Patient 0'])
valeurs = []
for elt in patients_concerned:
    valeurs = []
    for j in range(taille_Pathology):
        valeurs += [database_transfer['Pathology'][elt][j][0]]
    database_transfer['Pathology'][elt]=valeurs
    valeurs = []
    for j in range(taille_Localisation):
        valeurs += [database_transfer['Localisation'][elt][j][0]]
    database_transfer['Localisation'][elt]=valeurs
    valeurs = []
    for j in range(taille_Sex):
        valeurs += [database_transfer['Sex'][elt][j][0]]
    database_transfer['Sex'][elt]=valeurs
    valeurs = []

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
## Adaptation au OnehotEncoding

Variable = database_transfer["Pathology"].values
for elt in Column_txt:
    ### On récupère la matrice qui nous intéresse
    Variable = database_transfer[elt].values
    nb_lines = len(Variable)
    nb_columns = len(Variable[0])
    array = np.zeros((nb_lines,nb_columns))
    for i in range(nb_lines):
        for j in range(nb_columns):
            array[i,j] = Variable[i][j]
    ### On enlève la colonne sur laquelle on a travaillé
    database_transfer = database_transfer.drop(elt,axis=1)
    COLUMNS_index.remove(elt)
    ### 
    for index in range(nb_columns):
        COLUMNS_index = [elt+' variable %s' %(nb_columns-index)] + COLUMNS_index
    database_transfer = np.concatenate((array,database_transfer.values),axis=1)
    database_transfer = pd.DataFrame(database_transfer,index=patients_concerned,columns=COLUMNS_index)
    

In [15]:
## Test en prenant uniquement les images mais on garde la Pathology pour la cross_val en ShuffleSplit
for elt in database_transfer.columns:
    if (elt.startswith('Feature') or elt.startswith('Progression') or elt.startswith('Length') or elt.startswith('Width') or elt.startswith('Thickness') or elt.startswith('Volume')):
        pass
    else:
        database_transfer = database_transfer.drop(elt,axis=1)

Local_columns = database_transfer.columns
Local_columns = pd.Index.tolist(Local_columns) + ['Pathology']
database_transfer = database_transfer.values
database_transfer = np.concatenate((database_transfer,Label_shufflesplit.values.reshape(-1,1)),axis=1)
database_transfer = pd.DataFrame(database_transfer,index=patients_concerned,columns=Local_columns)
print(database_transfer.head())

          Progression    Length     Width  Thickness    Volume Feature Img1 1  \
Patient 0           1  0.254583   1.15921  -0.555554 -0.176102       0.937164   
Patient 1           0  -0.73829 -0.514587  -0.608432 -0.456753         0.5416   
Patient 3           1 -0.356416  0.179425 -0.0796446 -0.299721      -0.849653   
Patient 4           1   1.20927   0.71014    1.34808  0.583189      -0.516918   
Patient 5           1 -0.700102 -0.392114    -1.1901 -0.490323       -1.12529   

          Feature Img1 2 Feature Img1 3 Feature Img1 4 Feature Img1 5  \
Patient 0      -0.315606      -0.717776       0.222858       0.984394   
Patient 1       0.485979       -0.76115      -0.197364       0.431827   
Patient 3        1.11891       0.809193       -0.47729       -1.06491   
Patient 4       0.526831      -0.562769      -0.572812      -0.420074   
Patient 5      -0.912195      -0.396058      -0.834955      -0.496023   

                 ...         Feature Img2 2040 Feature Img2 2041  \
Patien

In [16]:
## Créer un jeu de test

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2,random_state=56)
# on le crée stratifié pour qu'il soit adapté aux différents types de cancer
for train, test in split.split(database_transfer,Label_shufflesplit):
    ####### -> train=index de ligne ms STIC_Transfer a les ID comme index #######
    train_index = ['' for k in range(len(train))]
    test_index = ['' for k in range(len(test))]
    for i in range(len(train)):
        train_index[i] = patients_concerned[train[i]]
    for i in range(len(test)):
        test_index[i] = patients_concerned[test[i]]

    database_transfer_train_set = database_transfer.loc[train_index]
    database_transfer_test_set = database_transfer.loc[test_index]
print(len(database_transfer_train_set),"train +",len(database_transfer_test_set),"test")
# Copier le jeu d'entraînement pour ne pas le déteriorer
database = database_transfer_train_set.copy()

129 train + 33 test


In [17]:
## Séparation jeu de données et labels

database_transfer_train_data = database_transfer_train_set.drop("Progression",axis=1)
database_transfer_train_data_copy = database_transfer_train_data.copy()
# drop crée une copie des données mais n'affecte pas le strat_train_set
database_transfer_train_labels = database_transfer_train_set["Progression"].copy()
database_transfer_test_data = database_transfer_test_set.drop("Progression",axis=1)
database_transfer_test_labels = database_transfer_test_set["Progression"].copy()

database_transfer_train_labels_1 = (database_transfer_train_labels==1).astype('int')
database_transfer_test_labels_1 = (database_transfer_test_labels==1).astype('int')

Bipper()

In [18]:
## Vérification : normalement il ne reste que la 'Progression', la 'Pathology' et les 4 variables de dimension

Columns_to_delete = []
for elt in database_transfer.columns:
    if elt.startswith('Feature'):
        pass
    else :
        Columns_to_delete += [elt]
len(Columns_to_delete)

6

## PARTIE 3 : Etude des données - Machine Learning

## Regression logistique

### Sans pénalité

In [19]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

log_reg = LogisticRegression()
log_reg.fit(database_transfer_train_data,database_transfer_train_labels_1)
log_reg.predict_proba(database_transfer_train_data)

accuracy = cross_val_score(log_reg, database_transfer_train_data, database_transfer_train_labels_1,
                           cv=3,scoring='accuracy')
print("Accuracy :",accuracy)
print("Mean Accuracy :",accuracy.mean())
print("Standard Accuracy :",accuracy.std())

AUC_score = cross_val_score(log_reg, database_transfer_train_data, database_transfer_train_labels_1,
                           cv=3,scoring='roc_auc')
print("Score AUC : ",AUC_score)
print("Mean Score AUC : ",AUC_score.mean())
print("Standard Score AUC : ",AUC_score.std())

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy : [0.36363636 0.51162791 0.42857143]
Mean Accuracy : 0.43461189972817876
Standard Accuracy : 0.06056808672292465


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Score AUC :  [0.48931624 0.60859729 0.34117647]
Mean Score AUC :  0.47969666499078273
Standard Score AUC :  0.10938578567586829


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Pénalité Ridge

In [20]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

max = 0
alphas = [100]
Boole = True
while Boole == True:
    Boole = False
    model_Ridge = RidgeCV(alphas)
    model_Ridge.fit(database_transfer_train_data,database_transfer_train_labels_1)
    # model.predict_proba(STIC_Transfer_train_data)
    AUC_score = cross_val_score(model_Ridge, database_transfer_train_data, database_transfer_train_labels_1,
                               cv=3,scoring='roc_auc')
    if AUC_score.mean() > max:
        max = AUC_score.mean()
        alphas[0] += 10
        Boole = True
print("alpha_max : ", alphas[0]-5)
print("Score AUC : ",AUC_score)
print("Mean Score AUC : ",AUC_score.mean())
print("Standard Score AUC : ",AUC_score.std())

alpha_max :  105
Score AUC :  [0.45454545 0.63657407 0.44827586]
Mean Score AUC :  0.5131317968961647
Standard Score AUC :  0.08732439061188707


### Pénalité Lasso

In [21]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

model_Lasso = LassoCV(max_iter=50,tol=0.001,alphas=[0.001],eps=0.01)
model_Lasso.fit(database_transfer_train_data,database_transfer_train_labels_1)
AUC_score = cross_val_score(model_Lasso, database_transfer_train_data, database_transfer_train_labels_1,
                           cv=3,scoring='roc_auc')
print("Score AUC : ",AUC_score)
print("Mean Score AUC : ",AUC_score.mean())
print("Standard Score AUC : ",AUC_score.std())

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not 

Score AUC :  [0.50865801 0.55092593 0.3817734 ]
Mean Score AUC :  0.48045244453290437
Standard Score AUC :  0.07187864646202032


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not 

### Pénalité ElasticNet

In [22]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

l1_ratios = [.1, .5, .7, .9, .95, .99, 1]
AUC_scores = {"0.1":"","0.5":"","0.7":"","0.9":"","0.95":"","0.99":"","1":""}
for elt in l1_ratios :
    model_elasticNet = ElasticNetCV(l1_ratio=elt,eps=0.01)
    AUC_score = cross_val_score(model_elasticNet, database_transfer_train_data, database_transfer_train_labels_1,
                               cv=3,scoring='roc_auc')
    AUC_scores[str(elt)]="AUC : %0.5f (+/- %0.5f)" %(AUC_score.mean(),AUC_score.std())
print(AUC_scores)

Bipper()

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv'

{'0.1': 'AUC : 0.44046 (+/- 0.04277)', '0.5': 'AUC : 0.44111 (+/- 0.05370)', '0.7': 'AUC : 0.44624 (+/- 0.05794)', '0.9': 'AUC : 0.44509 (+/- 0.06423)', '0.95': 'AUC : 0.45479 (+/- 0.07505)', '0.99': 'AUC : 0.45373 (+/- 0.07517)', '1': 'AUC : 0.44447 (+/- 0.06671)'}


### Descente de Gradient Stochastique et Recherche par quadrillage

In [23]:
##################################################
### Initialisation recherche par quadrillage
##################################################
param_grid = [{'max_iter':[k*50 for k in range(1,21)],'alpha':[0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1,5,10,50,100],
              'eta0':[10**k for k in range(-3,2)]}]
#param_grid =[{'max_iter':[10*k for k in range(1,10)],'alpha':[0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.002,0.003,0.004,0.005],
#              'eta0':[k for k in range(5,15)]}]
### On reste dans cet intervalle de valeurs car on sait qu'on y trouve un maximum local très proche 
### du possible maximum global
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

##################################################
### Pénalité LASSO
##################################################

param_grid = [{'max_iter':[845+k for k in range(10)],'alpha':[0.00045,0.00046,0.00047,0.00048,0.00049,0.0005,0.00051,0.00052,0.00053,0.00054],
              'eta0':[0.001*k for k in range(8,13)]}]
#sgd_reg_lasso = SGDRegressor(max_iter=400,penalty='l1',eta0=0.01, alpha=200)
#sgd_reg_lasso.fit(STIC_Transfer_train_data,STIC_Transfer_train_labels_1)
sgd_reg_lasso = SGDRegressor(penalty='l1', tol=0.001, max_iter=1000)
grid_search = GridSearchCV(sgd_reg_lasso,param_grid,cv=3,scoring='roc_auc')
grid_search.fit(database_transfer_train_data,database_transfer_train_labels_1)
cv_results = grid_search.cv_results_
print('---------------------------------------------------------------------------LASSO')
#for mean_score,std_score,params in zip(cv_results["mean_test_score"],cv_results["std_test_score"],cv_results["params"]):
#    print(mean_score,std_score,params)
#AUC_score_lasso = cross_val_score(sgd_reg_lasso, STIC_Transfer_train_data, STIC_Transfer_train_labels_1,
#                           cv=3,scoring='roc_auc')
#print("Score AUC....................:",cv_results["mean_test_score"][grid_search.best_params_], "[Lasso]")
print("Mean Score AUC...............:",cv_results["mean_test_score"][grid_search.best_index_], "[Lasso]")
print("Standard Score AUC...........:",cv_results["std_test_score"][grid_search.best_index_], "[Lasso]")
print("Best Params in Grid Search...:",grid_search.best_params_, "[Lasso]")

##################################################
### Pénalité RIDGE
##################################################

param_grid = [{'max_iter':[440+k for k in range(10)],'alpha':[0.45,0.46,0.47,0.48,0.49,0.5,0.51,0.52,0.53,0.54],
              'eta0':[0.001*k for k in range(8,13)],}]
#sgd_reg_ridge = SGDRegressor(max_iter=400,penalty='l2',eta0=0.01, alpha=200)
#sgd_reg_ridge.fit(STIC_Transfer_train_data,STIC_Transfer_train_labels_1)
sgd_reg_ridge = SGDRegressor(penalty='l2', tol=0.001, max_iter=1000)
grid_search = GridSearchCV(sgd_reg_ridge,param_grid,cv=3,scoring='roc_auc')
grid_search.fit(database_transfer_train_data,database_transfer_train_labels_1)
cv_results = grid_search.cv_results_
print('---------------------------------------------------------------------------RIDGE')
#for mean_score, params in zip(cv_results["mean_test_score"],cv_results["params"]):
#    print(mean_score,params)
#AUC_score_ridge = cross_val_score(sgd_reg_ridge, STIC_Transfer_train_data, STIC_Transfer_train_labels_1,
#                           cv=3,scoring='roc_auc')
#print("Score AUC....................:",AUC_score_ridge, "[Ridge]")
print("Mean Score AUC...............:",cv_results["mean_test_score"][grid_search.best_index_], "[Ridge]")
print("Standard Score AUC...........:",cv_results["std_test_score"][grid_search.best_index_], "[Ridge]")
print("Best Params in Grid Search...:",grid_search.best_params_, "[Ridge]")

##################################################
### Pénalité ELASTIC-NET
##################################################

param_grid = [{'max_iter':[k*10 for k in range(90,100)],'alpha':[0.01*k for k in range(5,15)],
              'eta0':[k for k in range(1,5)],'l1_ratio':[0.05*k for k in range(1,10)]}]

#sgd_reg_ElasticNet = SGDRegressor(max_iter=400,penalty='elasticnet',eta0=0.01, alpha=200)
#sgd_reg_ElasticNet.fit(STIC_Transfer_train_data,STIC_Transfer_train_labels_1)
sgd_reg_ElasticNet = SGDRegressor(penalty='elasticnet', tol=0.00001, max_iter=1000)
grid_search = GridSearchCV(sgd_reg_ElasticNet,param_grid,cv=3,scoring='roc_auc')
grid_search.fit(database_transfer_train_data,database_transfer_train_labels_1)
cv_results = grid_search.cv_results_
print('---------------------------------------------------------------------------ELASTICNET')
#for mean_score, params in zip(cv_results["mean_test_score"],cv_results["params"]):
#    print(mean_score,params)
#AUC_score_ElasticNet = cross_val_score(sgd_reg_ElasticNet, STIC_Transfer_train_data, STIC_Transfer_train_labels_1,
#                           cv=3,scoring='roc_auc')
#print("Score AUC....................:",AUC_score_ElasticNet, "[ElasticNet]")
print("Mean Score AUC...............:",cv_results["mean_test_score"][grid_search.best_index_], "[ElasticNet]")
print("Standard Score AUC...........:",cv_results["std_test_score"][grid_search.best_index_], "[ElasticNet]")
print("Best Params in Grid Search...:",grid_search.best_params_, "[ElasticNet]")


Bipper()

---------------------------------------------------------------------------LASSO
Mean Score AUC...............: 0.6562228746136792 [Lasso]
Standard Score AUC...........: 0.055200994395569754 [Lasso]
Best Params in Grid Search...: {'alpha': 0.00054, 'eta0': 0.01, 'max_iter': 847} [Lasso]
---------------------------------------------------------------------------RIDGE
Mean Score AUC...............: 0.6830642218573254 [Ridge]
Standard Score AUC...........: 0.11855935893504582 [Ridge]
Best Params in Grid Search...: {'alpha': 0.54, 'eta0': 0.012, 'max_iter': 445} [Ridge]


KeyboardInterrupt: 

Vérification du résultat avec ces paramètres

In [24]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

sgd_reg_lasso = SGDRegressor(penalty='l1', tol=0.001, max_iter=30,alpha=0.0009,eta0=11)
AUC_score_lasso = cross_val_score(sgd_reg_lasso, database_transfer_train_data, database_transfer_train_labels_1,
                                  cv=3,scoring='roc_auc')
print("Score AUC....................:",AUC_score_lasso, "[LASSO]")
print("Mean Score AUC...............:",AUC_score_lasso.mean(), "[LASSO]")
print("Standard Score AUC...........:",AUC_score_lasso.std(), "[LASSO]")

sgd_reg_ridge = SGDRegressor(penalty='l2', tol=0.001, max_iter=90,alpha=0.0008,eta0=6)
AUC_score_ridge = cross_val_score(sgd_reg_ridge, database_transfer_train_data, database_transfer_train_labels_1,
                                  cv=3,scoring='roc_auc')
print("Score AUC....................:",AUC_score_ridge, "[RIDGE]")
print("Mean Score AUC...............:",AUC_score_ridge.mean(), "[RIDGE]")
print("Standard Score AUC...........:",AUC_score_ridge.std(), "[RIDGE]")

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:1192: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:1192: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


Score AUC....................: [0.56709957 0.47916667 0.51231527] [LASSO]
Mean Score AUC...............: 0.5195271682340648 [LASSO]
Standard Score AUC...........: 0.03625885952990801 [LASSO]
Score AUC....................: [0.41558442 0.65509259 0.39655172] [RIDGE]
Mean Score AUC...............: 0.4890762441049797 [RIDGE]
Standard Score AUC...........: 0.1176481528677626 [RIDGE]


-> Change beaucoup quand on relance l'algo, le maximum se trouve toujours dans la même zone mais jamais avec exactement les mêmes paramètres

## Machines à Vecteurs de Support

### Classificateur SVM Linéaire

In [25]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

best_AUC = np.array([0])
best_C = 0
best_loss = ''
best_CV = 0
losses = ['hinge','squared_hinge']
Avancement = 0
for c in range(11):
    for loss in losses:
        for CV in range(3,11):
            svm_clf = Pipeline((
                    ("linear_svc", LinearSVC(C=0.001+c*0.0001,loss=loss)),
                ))
            AUC_Score = cross_val_score(svm_clf, database_transfer_train_data, database_transfer_train_labels_1,
                                       cv=CV,scoring='roc_auc')
            Avancement += 1/(11*2*8)
            print(Avancement)
            if AUC_Score.mean() > best_AUC.mean() :
                best_AUC = AUC_Score
                best_C = 0.001+c*0.0001
                best_loss = loss
                best_CV = CV

#svm_clf.fit(STIC_Transfer_train_data,STIC_Transfer_train_labels_1)
#svm_clf.predict(STIC_Transfer_train_data)

#print("Score AUC : ",AUC_Score)
#print("Mean Score AUC : ",AUC_Score.mean())
#print("Standard Score AUC : ",AUC_Score.std())

print("AUC optimisée........:",best_AUC)
print("Mean AUC optimisée...:",best_AUC.mean())
print(" ")
print("Best C...............:",best_C)
print("Best Loss Function...:",best_loss)
print("Cross Validation.....:",best_CV)

Bipper()

0.005681818181818182
0.011363636363636364
0.017045454545454544
0.022727272727272728
0.02840909090909091
0.034090909090909095
0.03977272727272728
0.04545454545454546
0.051136363636363646
0.05681818181818183
0.06250000000000001
0.06818181818181819
0.07386363636363637
0.07954545454545454
0.08522727272727272
0.0909090909090909
0.09659090909090907
0.10227272727272725
0.10795454545454543
0.1136363636363636
0.11931818181818178
0.12499999999999996
0.13068181818181815
0.13636363636363633
0.1420454545454545
0.14772727272727268
0.15340909090909086
0.15909090909090903
0.1647727272727272
0.1704545454545454
0.17613636363636356
0.18181818181818174
0.18749999999999992
0.1931818181818181
0.19886363636363627
0.20454545454545445
0.21022727272727262
0.2159090909090908
0.22159090909090898
0.22727272727272715
0.23295454545454533
0.2386363636363635
0.24431818181818168
0.24999999999999986
0.25568181818181807
0.26136363636363624
0.2670454545454544
0.2727272727272726
0.2784090909090908
0.28409090909090895
0.289

    -> SVM_lin > Reg_Log : vrai avant plus maintenant

### Classificateur SVM Non Linéaire

In [26]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

polynomial_svm_clf = Pipeline((
        ("poly_features",PolynomialFeatures(degree=1)),
        ("svc", SVC(kernel='poly',C=1,degree=5)),#,loss="hinge")),
    ))
#polynomial_svm_clf.fit(STIC_Transfer_train_data,STIC_Transfer_train_labels_1)
#polynomial_svm_clf.predict(STIC_Transfer_train_data)

AUC_Score = cross_val_score(polynomial_svm_clf, database_transfer_train_data, database_transfer_train_labels_1,
                           cv=5,scoring='roc_auc')
print("Score AUC : ", AUC_Score)
print("Mean Score AUC : ", AUC_Score.mean())
print("Standard Score AUC : ", AUC_Score.std())

Bipper()

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/

Score AUC :  [0.4375     0.39772727 0.6        0.44       0.58      ]
Mean Score AUC :  0.4910454545454545
Standard Score AUC :  0.08241970798578949


    -> A peine meilleur

#### SVM Non Linéaire Avec l'astuce du noyau

##### Noyau Polynomial

In [27]:
# Mieux de normaliser pour être moins sensible aux valeurs extrêmes que transfo min-max
standardiser = StandardScaler()

database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)
lines = database_transfer_train_data.index
columns = database_transfer_train_data.columns

database_transfer_train_data = standardiser.fit_transform(database_transfer_train_data)    

database_transfer_train_data = pd.DataFrame(database_transfer_train_data,index=lines,columns=columns)

#poly_kernel_svm_clf = Pipeline((
#        ("scaler1",StandardScaler()),
#        #("poly_features",PolynomialFeatures(degree=1))
#        ("svm_clf", SVC(kernel="poly",degree=7,coef0=0.0001,C=1)),
#    ))
#poly_kernel_svm_clf.fit(STIC_Transfer_train_data,STIC_Transfer_train_labels_1)
#poly_kernel_svm_clf.predict(STIC_Transfer_train_data)

param_grid = [{'max_iter':[k for k in range(8,15)],'degree':[k for k in range(1,8)],
              'C':[3+0.1*k for k in range(10)]}]
#param_grid =[{'max_iter':[10*k for k in range(1,10)],'alpha':[0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.002,0.003,0.004,0.005],
#              'eta0':[k for k in range(5,15)],}]
### On reste dans cet intervalle de valeurs car on sait qu'on y trouve un maximum local très proche 
### du possible maximum global
svm_clf = SVC(kernel="poly",degree=5,coef0=0.0,C=1,max_iter=-1)
grid_search = GridSearchCV(svm_clf,param_grid,cv=4,scoring='roc_auc')
grid_search.fit(database_transfer_train_data,database_transfer_train_labels_1)
cv_results = grid_search.cv_results_
#for mean_score,std_score,params in zip(cv_results["mean_test_score"],cv_results["std_test_score"],cv_results["params"]):
#    print(mean_score,std_score,params)
#AUC_score = cross_val_score(svm_clf, STIC_Transfer_train_data, STIC_Transfer_train_labels_1,
#                           cv=3,scoring='roc_auc')
#print("Score AUC....................:",cv_results["mean_test_score"][grid_search.best_params_], "[Lasso]")
print("Mean Score AUC...............:",cv_results["mean_test_score"][grid_search.best_index_], "[SVC]")
print("Standard Score AUC...........:",cv_results["std_test_score"][grid_search.best_index_], "[SVC]")
print("Best Params in Grid Search...:",grid_search.best_params_, "[SVC]")


#AUC_Score = cross_val_score(poly_kernel_svm_clf, STIC_Transfer_train_data, STIC_Transfer_train_labels_1,
#                           cv=4,scoring='roc_auc')
#print("Score AUC : ",AUC_Score)
#print("Mean Score AUC : ",AUC_Score.mean())
#print("Standard Score AUC : ",AUC_Score.std())

Bipper()

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=8).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarni

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=11).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=11).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better fo

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=9).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=13).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=13).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better fo

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=9).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for

...

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=14).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early 

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=12).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early 

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=9).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=13).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=13).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better fo

Mean Score AUC...............: 0.5891708250949376 [SVC]
Standard Score AUC...........: 0.0281770423797001 [SVC]
Best Params in Grid Search...: {'C': 3.0, 'degree': 1, 'max_iter': 12} [SVC]


##### Noyau radial gaussien

In [28]:
database_transfer_train_data = database_transfer_train_data_copy.drop('Pathology',axis=1)

rbf_kernel_svm_clf = Pipeline((
        ("scaler",StandardScaler()),
        ("svm_clf", SVC(kernel="rbf",gamma=0.000001,coef0=0.0,C=1,max_iter=100)),
    ))
rbf_kernel_svm_clf.fit(database_transfer_train_data,database_transfer_train_labels_1)
rbf_kernel_svm_clf.predict(database_transfer_train_data)

AUC_Score = cross_val_score(rbf_kernel_svm_clf, database_transfer_train_data, database_transfer_train_labels_1,
                           cv=3,scoring='roc_auc')
print("Score AUC : ",AUC_Score)
print("Mean Score AUC : ",AUC_Score.mean())
print("Standard Score AUC : ",AUC_Score.std())

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype object w

Score AUC :  [0.53418803 0.56108597 0.33882353]
Mean Score AUC :  0.47803251215015924
Standard Score AUC :  0.0990462196228569


/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Applications/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:401: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


###### Test quand la cross val est en ShuffleSplit sur la 'Pathology' -> à tester avec SVM et LogReg mais pas AdaBoost

In [29]:
save_clf = Pipeline((
        #("scaler",StandardScaler()),
        ("poly_features1",PolynomialFeatures(degree=1)),
        #("poly_features2",PolynomialFeatures(degree=2)),
        ("AdaBoost", AdaBoostClassifier(DecisionTreeClassifier(max_depth=3, min_samples_leaf=5),
                                   n_estimators=100,algorithm="SAMME.R", learning_rate=0.2)),
    ))

CV = 3

AUC_mean_boucle = np.array([0.0 for k in range(5)])
for boucle in range(5):
    database_transfer_train_data=database_transfer_train_data_copy
    sss = StratifiedShuffleSplit(n_splits=CV, test_size=0.2, random_state=42)
    sss = sss.split(database_transfer_train_data, database_transfer_train_data['Pathology'])
    #########################################
    ### On retire 'Pathology' des variables
    #########################################
    database_transfer_train_data = database_transfer_train_data.drop("Pathology",axis=1)
    rank = 0
    AUC_Scores = np.array([0.0 for k in range(CV)])
    for train_index, test_index in sss:
        print("----------------------------------------------------------------------------------RANK:",rank)
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = database_transfer_train_data.values[train_index], database_transfer_train_data.values[test_index]
        y_train, y_test = database_transfer_train_labels_1.values[train_index], database_transfer_train_labels_1.values[test_index]
        save_clf.fit(X_train,y_train)
        y_score = save_clf.predict(X_test)
        AUC_Score = roc_auc_score(y_true=y_test,y_score=y_score)
        #AUC_Score = accuracy_score(y_true=y_test,y_pred=y_score)
        AUC_Scores[rank] += AUC_Score
        rank += 1
    print(AUC_Scores.mean())
    AUC_mean_boucle[boucle] = AUC_Scores.mean()
    
#Accuracy = cross_val_score(ada_clf, STIC_Transfer_train_data, STIC_Transfer_train_labels_1,
#                           cv=6,scoring='accuracy')

print("Score AUC............:",AUC_mean_boucle)
print("Mean Score AUC.......:",AUC_mean_boucle.mean())
print("Standard Score AUC...:",AUC_mean_boucle.std())

Bipper()

----------------------------------------------------------------------------------RANK: 0
----------------------------------------------------------------------------------RANK: 1
----------------------------------------------------------------------------------RANK: 2
0.5916847972575526
----------------------------------------------------------------------------------RANK: 0
----------------------------------------------------------------------------------RANK: 1
----------------------------------------------------------------------------------RANK: 2
0.5824580762661258
----------------------------------------------------------------------------------RANK: 0
----------------------------------------------------------------------------------RANK: 1
----------------------------------------------------------------------------------RANK: 2
0.5549406790892859
----------------------------------------------------------------------------------RANK: 0
-------------------------------------------

In [ ]:
# database_STIC = database_transfer_copy.loc[:,["Pathology", "Localisation", "Length", "Width", "Thickness", "Sex", "Progression", "Volume"]]
# database_STIC.to_pickle("database_STIC.pkl")